# DISPLAY MAPS

## Display sea ice statistics maps in the Weddell Sea using .nc files
- Read netCDF files for the sea ice statistics
- Draw sea ice statistics maps in the Weddell Sea

Credited by Younghyun Koo (kooala317@gmail.com)

## (1) Import necessary libraries

In [1]:
import warnings
warnings.filterwarnings('ignore')

import os, glob
import numpy as np
import h5py
import matplotlib.pylab as plt
from math import *
import netCDF4
from netCDF4 import date2num,num2date
import pandas as pd
import cartopy.crs as ccrs
import datetime as dt

## (2) Read .nc file

In [9]:
# Enter name of the .nc file
lead_type = "S"
ncname = f'D:\\Floes\\array\\Grid_fb_v6_entire_{lead_type}.nc'

with netCDF4.Dataset(ncname, 'r') as nc:
    keys = nc.variables.keys()
    print(keys)
    
    # X/Y coordinates and lat/lon on the NSIDC sea ice stereogeographic south
    x = np.array(nc.variables['x'])
    y = np.array(nc.variables['y'])
    xx, yy = np.meshgrid(x, y)
    lat = np.array(nc.variables['lat'])
    lon = np.array(nc.variables['lon'])
    
    # Times in seconds from 1800/01/01
    times = np.array(nc.variables['time'])
    t = num2date(nc.variables['time'], nc.variables['time'].units)
    
    # Freeboard statistics
    fmode = np.array(nc.variables['fb_mode'])
    fmean = np.array(nc.variables['fb_mean'])
    fmed = np.array(nc.variables['fb_med'])
    fstd = np.array(nc.variables['fb_std'])
    fridge = np.array(nc.variables['fr_ridge'])
    hridge = np.array(nc.variables['h_ridge'])
    flead = np.array(nc.variables['fr_lead'])
    
    # Floe statistics
    flen = np.array(nc.variables['floe_len'])
    fcnt = np.array(nc.variables['floe_cnt'])

dict_keys(['x', 'y', 'time', 'lat', 'lon', 'fb_mode', 'fb_mean', 'fb_med', 'fb_std', 'fr_ridge', 'h_ridge', 'fr_lead', 'floe_len', 'floe_cnt'])


## (3) Draw maps in the Weddell Sea

In [10]:
# Draw the mean of the variable in the time series from date1 to date2
variable = "Fr_lead"
for year in [2018, 2019, 2020, 2021, 2022]:
    if year == 2018:
        months = np.array([10, 11, 12])
    elif year == 2023:
        months = np.array([1, 2, 3])
    else:
        months = np.arange(1, 13)
    
    for month in months:
        date1 = dt.datetime(year, month, 1)
        date2 = date1 + dt.timedelta(days = 60)

        fig, ax = fig, ax = plt.subplots(1, 1, figsize=(6,6), dpi= 100, subplot_kw={'projection': ccrs.SouthPolarStereo()})
        ax.coastlines()
        idx = (t >= date1) & (t < date2)
        
        if variable == "Fmode":        
            m = ax.pcolormesh(xx, yy, np.nanmean(fmode[idx], axis=0), vmin=0, vmax=1)
        elif variable == "Fmean":        
            m = ax.pcolormesh(xx, yy, np.nanmean(fmean[idx], axis=0), vmin=0, vmax=1)
        elif variable == "Fstd":
            m = ax.pcolormesh(xx, yy, np.nanmean(fstd[idx], axis=0), vmin=0, vmax=0.5)
        elif variable == "Floe_len":
            m = ax.pcolormesh(xx, yy, np.nanmean(np.log10(flen[idx]), axis=0), vmin=np.log10(10), vmax=np.log10(10000))
        elif variable == "FL_ratio":
            m = ax.pcolormesh(xx, yy, np.nanmean(fmean[idx]/(flen[idx]/1000), axis=0), vmin=0)
        elif variable == "Fr_lead":
            m = ax.pcolormesh(xx, yy, np.nanmean(flead[idx], axis=0), vmin=0, vmax = 20, cmap = "jet")
        fig.colorbar(m, ax=ax, shrink = 0.5)
        ax.set_title(f"{variable} {str(int(year*100+month))}")
        plt.savefig(f"D:\\Floes\\figures\\{variable}_{lead_type}_{str(int(year*100+month))}.png", bbox_inches = "tight")
        plt.close()

## Generate video

In [21]:
import cv2
import os

for var in ["Fmode", "Fstd", "Floe_len"]:
    image_folder = 'D:\\Floes\\figures'
    video_name = image_folder + f'\\{var}.avi'

    images = glob.glob(image_folder + f"\\{var}_*.png") #[img for img in os.listdir(image_folder) if img.endswith(".png")]
    frame = cv2.imread(os.path.join(image_folder, images[0]))
    height, width, layers = frame.shape

    video = cv2.VideoWriter(video_name, 0, 1, (width,height))

    for image in images:
        video.write(cv2.imread(os.path.join(image_folder, image)))

    cv2.destroyAllWindows()
    video.release()